In [96]:
import pandas as pd
from dont_patronize_me import DontPatronizeMe
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

RANDOM_STATE = 42

In [57]:

train_path = ".\\"
test_path = ".\\data\\dev_semeval_parids-labels.csv"
dpm = DontPatronizeMe(train_path, test_path)

In [88]:
dpm.load_task1()
#dpm.load_test()

In [89]:
data_set = dpm.train_task1_df
texts = data_set['text']
labels = data_set['label']

In [99]:


# Load a pre-trained Word2Vec model (e.g., Google News vectors)
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

# Function to convert text to Word2Vec vector
def text_to_word2vec_vector(text):
    words = text.split()
    word_vectors = [word_vectors[word] for word in words if word in word_vectors]
    if not word_vectors:
        return np.zeros(word_vectors.vector_size)
    return np.mean(word_vectors, axis=0)

# Convert your texts to Word2Vec features
X_word2vec = np.array([text_to_word2vec_vector(text) for text in texts])

NameError: free variable 'word_vectors' referenced before assignment in enclosing scope

In [90]:
# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the model and transform the texts to a BoW representation
X = vectorizer.fit_transform(texts)

X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.2, random_state=RANDOM_STATE)

In [91]:
# Initialize the classifier
bow_clf = LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE)

# Train the classifier
bow_clf.fit(X_train, y_train)


c:\Users\Lance Bao\anaconda3\envs\HITL\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=42)

In [92]:
# Make predictions on the test set
predictions = bow_clf.predict(X_val)

# Calculate the accuracy
f1 = f1_score(y_val, predictions)
print(f"F1: {f1}")
# Print a classification report to see precision, recall, and F1-score
print(classification_report(y_val, predictions))


F1: 0.406392694063927
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      1883
           1       0.39      0.42      0.41       211

    accuracy                           0.88      2094
   macro avg       0.66      0.67      0.67      2094
weighted avg       0.88      0.88      0.88      2094



In [93]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000, )

X_tfidf = tfidf_vectorizer.fit_transform(texts)
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Initialize and train the classifier
tfidf_clf = LogisticRegression(random_state=42, class_weight='balanced')
tfidf_clf.fit(X_train, y_train)



LogisticRegression(class_weight='balanced', random_state=42)

In [94]:
# Make predictions on the test set
predictions = tfidf_clf.predict(X_val)

# Calculate the accuracy
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy}")
f1 = f1_score(y_val, predictions)
print(f"F1: {f1}")
# Print a classification report to see precision, recall, and F1-score
print(classification_report(y_val, predictions))

Accuracy: 0.8553008595988538
F1: 0.41392649903288203
              precision    recall  f1-score   support

           0       0.94      0.89      0.92      1883
           1       0.35      0.51      0.41       211

    accuracy                           0.86      2094
   macro avg       0.65      0.70      0.67      2094
weighted avg       0.88      0.86      0.87      2094



In [66]:
# True Positive samples
# true_positive = texts[(predictions == 1)]
# true_positive